## Installation (CAN SKIP ONLY NEED TO INSTALL ONCE INTO THIS NOTEBOOK INSTANCE)

Install the latest version of Vertex AI SDK for Python.

In [128]:
import os

# # Google Cloud Notebook
# if os.path.exists("/opt/deeplearning/metadata/env_version"):
#     USER_FLAG = "--user"
# else:
#     USER_FLAG = ""

# ! pip3 install --upgrade google-cloud-aiplatform $USER_FLAG

Install the latest GA version of *google-cloud-storage* library as well.

In [129]:
# ! pip3 install -U google-cloud-storage $USER_FLAG

Install the latest GA version of *google-cloud-pipeline-components* library as well.

In [130]:
# ! pip3 install $USER kfp google-cloud-pipeline-components --upgrade

### Restart the kernel

Once you've installed the additional packages, you need to restart the notebook kernel so it can find the packages.

In [4]:
import os

# if not os.getenv("IS_TESTING"):
#     # Automatically restart kernel after installs
#     import IPython

#     app = IPython.Application.instance()
#     app.kernel.do_shutdown(True)

Check the versions of the packages you installed.  The KFP SDK version should be >=1.6.

In [1]:
! python3 -c "import kfp; print('KFP SDK version: {}'.format(kfp.__version__))"
! python3 -c "import google_cloud_pipeline_components; print('google_cloud_pipeline_components version: {}'.format(google_cloud_pipeline_components.__version__))"

KFP SDK version: 1.8.12
google_cloud_pipeline_components version: 1.0.8


## START RUNNING HERE

## Before you begin

### GPU runtime

This tutorial does not require a GPU runtime.

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

2. [Make sure that billing is enabled for your project.](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [Enable the Vertex AI APIs, Compute Engine APIs, and Cloud Storage.](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component,storage-component.googleapis.com)

4. [The Google Cloud SDK](https://cloud.google.com/sdk) is already installed in Google Cloud Notebook.

5. Enter your project ID in the cell below. Then run the  cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$`.

In [12]:
PROJECT_ID = "daring-hash-348101"  # @param {type:"string"}

In [13]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [4]:
# this step causes bash to ask for reply, which cannot be done in notebook. So use cloud shell instead
# but no need to do it if project is already set to the right one

# ! gcloud config set project $PROJECT_ID

#### Region

You can also change the `REGION` variable, which is used for operations
throughout the rest of this notebook.  Below are regions supported for Vertex AI. We recommend that you choose the region closest to you.

- Americas: `us-central1`
- Europe: `europe-west4`
- Asia Pacific: `asia-east1`

You may not use a multi-regional bucket for training with Vertex AI. Not all regions provide support for all Vertex AI services.

Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations)

In [14]:
REGION = "us-east1"  # @param {type: "string"}

#### Timestamp

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a timestamp for each instance session, and append the timestamp onto the name of resources you create in this tutorial.

In [15]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**

When you initialize the Vertex AI SDK for Python, you specify a Cloud Storage staging bucket. The staging bucket is where all the data associated with your dataset and model resources are retained across sessions.

Set the name of your Cloud Storage bucket below. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.

In [16]:
BUCKET_NAME = "seangoh-smu-mle-usa"
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [17]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "gs://[your-bucket-name]":
    BUCKET_NAME = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [42]:
! gsutil mb -l $REGION $BUCKET_NAME

CommandException: "mb" command does not support "file://" URLs. Did you mean to use a gs:// URL?


Finally, validate access to your Cloud Storage bucket by examining its contents:

In [18]:
! gsutil ls -al $BUCKET_URI

                                 gs://seangoh-smu-mle-usa/FaceMask/
                                 gs://seangoh-smu-mle-usa/Models/
                                 gs://seangoh-smu-mle-usa/logs/


#### Service Account

**If you don't know your service account**, try to get your service account using `gcloud` command by executing the second cell below.

**LEAVE THIS AS `[your-service-account]` AND LET GCLOUD COMMAND FIGURE IT OUT**

In [19]:
SERVICE_ACCOUNT = "[your-service-account]"  # @param {type:"string"}

In [20]:
if (
    SERVICE_ACCOUNT == ""
    or SERVICE_ACCOUNT is None
    or SERVICE_ACCOUNT == "[your-service-account]"
):
    # Get your GCP project id from gcloud
    shell_output = !gcloud auth list 2>/dev/null
    SERVICE_ACCOUNT = shell_output[2].replace('*', '').strip()
    print("Service Account:", SERVICE_ACCOUNT)

Service Account: 591661299323-compute@developer.gserviceaccount.com


#### Set service account access for Vertex AI Pipelines

Run the following commands to grant your service account access to read and write pipeline artifacts in the bucket that you created in the previous step -- you only need to run these once per service account.

In [9]:
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectCreator $BUCKET_URI

! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectViewer $BUCKET_URI

### Set up variables

Next, set up some variables used throughout the tutorial.
### Import libraries and define constants

In [21]:
import google.cloud.aiplatform as aip
from google_cloud_pipeline_components import aiplatform as gcc_aip
from google_cloud_pipeline_components.experimental.custom_job import utils
from google_cloud_pipeline_components.v1.model import \
    ModelUploadOp as model_upload_op

import kfp
from kfp.v2 import dsl, compiler
from kfp.v2.components import importer_node
from kfp.v2.dsl import (Artifact,
                        Dataset,
                        Input,
                        Model,
                        Output,
                        Metrics,
                        ClassificationMetrics,
                        component, 
                        OutputPath, 
                        InputPath)
from typing import NamedTuple

import tensorflow as tf

#### Vertex AI Pipelines constants

Setup up the following constants for Vertex AI Pipelines:

`PIPELINE_ROOT`: The artifact repository where KFP stores a pipeline’s artifacts. <br>
`CONTAINER_IMG_URI`: Container image for training

In [22]:
PIPELINE_ROOT = "{}/pipeline_root/face-mask".format(BUCKET_URI)
# CONTAINER_IMG_URI = 'us-east1-docker.pkg.dev/daring-hash-348101/smu-mle-usa/efficientnettrain:latest'
CONTAINER_IMG_URI = 'us-east1-docker.pkg.dev/daring-hash-348101/smu-mle-usa/mobilenettrain:latest'
MODEL_URI = "gs://seangoh-smu-mle-usa/Models/FaceMaskMobileNetModel/" # change to cloud storage stored model location
DISPLAY_NAME = "face-mask-classification-pipeline_" + TIMESTAMP
TEST_URI = "" # change to cloud storage test dataset location
MODEL_DISPLAY_NAME = f"face-mask-classification-{TIMESTAMP}"
THRESHOLDS_DICT_STR = "{'f1': 0.8}"

## Initialize Vertex AI SDK for Python

Initialize the Vertex AI SDK for Python for your project and corresponding bucket.

In [23]:
aip.init(
    project=PROJECT_ID, 
    staging_bucket=BUCKET_URI,
    location=REGION,
)

## Define image classification model pipeline that uses components from `google_cloud_pipeline_components` and custom components

Next, you define the pipeline.

<img alt="alt_text" width="1000px" src="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/community-content/pytorch_text_classification_using_vertex_sdk_and_gcloud/images/pipelines-high-level-flow.png?raw=true" />

## Create the pipeline

Next, create the pipeline.

<img alt="alt_text" width="1000px" src="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/community-content/pytorch_text_classification_using_vertex_sdk_and_gcloud/images/concepts-of-a-pipeline.png?raw=true" />


#### This is new and experimental `custom_job_wrapper` to run any component as a custom component. Can't get it to work so don't run this portion. Skip to the next


In [15]:
# # CREATE PIPELINE
# from google_cloud_pipeline_components.experimental import custom_job

# @kfp.dsl.pipeline(
#     name="face-mask-pipeline-v1",
#     pipeline_root=PIPELINE_ROOT,
# )
# def pipeline(
#     project: str, 
#     region: str,
#     service_account: str,
#     container_img_uri: str
# ):
    
#     # RUN TRAINING (CUSTOM COMPONENT FOR RUNNING CONTAINER)
    
#     # ========================================================================
#     # model training
#     # ========================================================================
#     # train the model on Vertex AI by submitting a CustomJob
#     # using the custom container (no hyper-parameter tuning)
#     # define training code arguments
#     # training_args = ["--num-epochs", "2", "--model-name", cfg.MODEL_NAME]
    
#     # define job name
#     JOB_NAME = f"face-mask-train-{TIMESTAMP}"
#     GCS_BASE_OUTPUT_DIR = f"{PIPELINE_ROOT}/mobilenett/"
    
#     # define worker pool specs
#     worker_pool_specs = [
#         {
#             "machine_spec": {
#                 "machine_type": 'n1-standard-8',
#                 "accelerator_type": 'NVIDIA_TESLA_K80',
#                 "accelerator_count": '1',
#             },
#             "replica_count": '1',
#             "container_spec": {"image_uri": f'{container_img_uri}'},
#         }
#     ]
    
    
#     run_train_task = (
#         custom_job.CustomTrainingJobOp(
#             project=f'{project}',
#             location=f'{region}',
#             display_name=JOB_NAME,
#             base_output_directory=GCS_BASE_OUTPUT_DIR,
#             worker_pool_specs=worker_pool_specs,
#         )
#         .set_display_name("Run custom training job")
#         .after(build_custom_train_image_task)
#     )

## Compile the pipeline

Next, compile the pipeline.

In [16]:
# from kfp.v2 import compiler  # noqa: F811

# compiler.Compiler().compile(
#     pipeline_func=pipeline,
#     package_path="image classification_pipeline.json".replace(" ", "_"),
# )

## Run the pipeline

Next, run the pipeline.

In [17]:
# pipeline_params = {
#     "project": PROJECT_ID,
#     "region": REGION,
#     "service_account": SERVICE_ACCOUNT,
#     "container_img_uri": CONTAINER_IMG_URI,
# }
# DISPLAY_NAME = "face_mask_pipeline_" + TIMESTAMP

# aip.init(
#     project=PROJECT_ID,
#     staging_bucket=BUCKET_URI,
#     location=REGION,
# )

# job = aip.PipelineJob(
#     display_name=DISPLAY_NAME,
#     template_path="image classification_pipeline.json".replace(" ", "_"),
#     pipeline_root=PIPELINE_ROOT,
#     enable_caching=False,
#     parameter_values=pipeline_params,
# )
# job.run()

#### USE THIS PORTION. This method is going to be deprecated by GCP but it works for now.


## Create the pipeline

Next, create the pipeline.

In [36]:
from google_cloud_pipeline_components.types import artifact_types
from google_cloud_pipeline_components.v1.endpoint import (EndpointCreateOp,
                                                              ModelDeployOp)
from google_cloud_pipeline_components.v1.model import ModelUploadOp

In [49]:
aip.init(
    project=PROJECT_ID,
    staging_bucket=BUCKET_URI,
    location=REGION,
)

In [50]:
@kfp.dsl.pipeline(
    name="face-mask-import-model-v1",
    pipeline_root=PIPELINE_ROOT,
)
def pipeline(
    display_name:str,
    project: str, 
    region: str,
    bucket: str,
    container_img_uri: str,
    artifact_uri: str,
):
    
    # custom model training for custom container
    training_op = gcc_aip.CustomContainerTrainingJobRunOp(
        display_name=display_name,
        container_uri=container_img_uri,
        project=project,
        location=region,
        staging_bucket=bucket,
        machine_type="n1-standard-4",
        replica_count=1,
        accelerator_type='NVIDIA_TESLA_K80',
        accelerator_count=1,  
    )
    
    # import the model from GCS
    import_unmanaged_model_task = importer_node.importer(
            artifact_uri=artifact_uri,
            artifact_class=artifact_types.UnmanagedContainerModel,
            metadata={
                "containerSpec": {
                    "imageUri": "us-docker.pkg.dev/cloud-aiplatform/prediction/tf2-gpu.2-8:latest",
                },
            },
        )
    import_unmanaged_model_task.after(training_op)
    
    # upload the model to VertexAI Model Registry
    model_upload_op = ModelUploadOp(
        project=project,
        location=region,
        display_name=display_name,
        unmanaged_container_model=import_unmanaged_model_task.outputs["artifact"],
    )
    model_upload_op.after(import_unmanaged_model_task)

    # create Endpoint. This is run in parallel as it is not dependent on previous nodes
    endpoint_create_op = EndpointCreateOp(
        project=project,
        location=region,
        display_name="pipelines-created-endpoint",
    )
    
    # deploy the model from Model Registry
    ModelDeployOp(
        endpoint=endpoint_create_op.outputs["endpoint"],
        model=model_upload_op.outputs["model"],
        deployed_model_display_name=display_name,
        dedicated_resources_machine_type="n1-standard-4",
        dedicated_resources_accelerator_type="NVIDIA_TESLA_K80",
        dedicated_resources_accelerator_count=1,
        dedicated_resources_min_replica_count=1,
        dedicated_resources_max_replica_count=1,
    )

## Compile the pipeline

Next, compile the pipeline.

In [51]:
compiler.Compiler().compile(
    pipeline_func=pipeline,
    package_path="facemask classification pipeline.json".replace(" ", "_"),
)

## Run the pipeline

Next, run the pipeline.

In [52]:
pipeline_params = {
    "display_name": DISPLAY_NAME,
    "project": PROJECT_ID,
    "region": REGION,
    "bucket": BUCKET_URI,
    "container_img_uri": CONTAINER_IMG_URI,
    "artifact_uri": MODEL_URI,
}

job = aip.PipelineJob(
    display_name=DISPLAY_NAME,
    template_path="facemask classification pipeline.json".replace(" ", "_"),
    pipeline_root=PIPELINE_ROOT,
    enable_caching=False,
    parameter_values=pipeline_params,
)

job.run()

Creating PipelineJob
PipelineJob created. Resource name: projects/591661299323/locations/us-east1/pipelineJobs/face-mask-import-model-v1-20220613112911
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/591661299323/locations/us-east1/pipelineJobs/face-mask-import-model-v1-20220613112911')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-east1/pipelines/runs/face-mask-import-model-v1-20220613112911?project=591661299323
PipelineJob projects/591661299323/locations/us-east1/pipelineJobs/face-mask-import-model-v1-20220613112911 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/591661299323/locations/us-east1/pipelineJobs/face-mask-import-model-v1-20220613112911 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/591661299323/locations/us-east1/pipelineJobs/face-mask-import-model-v1-20220613112911 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/59166129

Click on the generated link to see your run in the Cloud Console.

<!-- It should look something like this as it is running:

<a href="https://storage.googleapis.com/amy-jo/images/mp/automl_tabular_classif.png" target="_blank"><img src="https://storage.googleapis.com/amy-jo/images/mp/automl_tabular_classif.png" width="40%"/></a> -->

In the UI, many of the pipeline DAG nodes will expand or collapse when you click on them. Here is a partially-expanded view of the DAG (click image to see larger version).

<a href="https://storage.googleapis.com/amy-jo/images/mp/automl_image_classif.png" target="_blank"><img src="https://storage.googleapis.com/amy-jo/images/mp/automl_image_classif.png" width="40%"/></a>

# Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial -- *Note:* this is auto-generated and not all resources may be applicable for this tutorial:

- Dataset
- Pipeline
- Model
- Endpoint
- Batch Job
- Custom Job
- Hyperparameter Tuning Job
- Cloud Storage Bucket

In [ ]:
# delete_dataset = True
# delete_pipeline = True
# delete_model = True
# delete_endpoint = True
# delete_batchjob = True
# delete_customjob = True
# delete_hptjob = True
# delete_bucket = True

# try:
#     if delete_model and "DISPLAY_NAME" in globals():
#         models = aip.Model.list(
#             filter=f"display_name={DISPLAY_NAME}", order_by="create_time"
#         )
#         model = models[0]
#         aip.Model.delete(model)
#         print("Deleted model:", model)
# except Exception as e:
#     print(e)

# try:
#     if delete_endpoint and "DISPLAY_NAME" in globals():
#         endpoints = aip.Endpoint.list(
#             filter=f"display_name={DISPLAY_NAME}_endpoint", order_by="create_time"
#         )
#         endpoint = endpoints[0]
#         endpoint.undeploy_all()
#         aip.Endpoint.delete(endpoint.resource_name)
#         print("Deleted endpoint:", endpoint)
# except Exception as e:
#     print(e)

# if delete_dataset and "DISPLAY_NAME" in globals():
#     if "image" == "tabular":
#         try:
#             datasets = aip.TabularDataset.list(
#                 filter=f"display_name={DISPLAY_NAME}", order_by="create_time"
#             )
#             dataset = datasets[0]
#             aip.TabularDataset.delete(dataset.resource_name)
#             print("Deleted dataset:", dataset)
#         except Exception as e:
#             print(e)

#     if "image" == "image":
#         try:
#             datasets = aip.ImageDataset.list(
#                 filter=f"display_name={DISPLAY_NAME}", order_by="create_time"
#             )
#             dataset = datasets[0]
#             aip.ImageDataset.delete(dataset.resource_name)
#             print("Deleted dataset:", dataset)
#         except Exception as e:
#             print(e)

#     if "image" == "text":
#         try:
#             datasets = aip.TextDataset.list(
#                 filter=f"display_name={DISPLAY_NAME}", order_by="create_time"
#             )
#             dataset = datasets[0]
#             aip.TextDataset.delete(dataset.resource_name)
#             print("Deleted dataset:", dataset)
#         except Exception as e:
#             print(e)

#     if "image" == "video":
#         try:
#             datasets = aip.VideoDataset.list(
#                 filter=f"display_name={DISPLAY_NAME}", order_by="create_time"
#             )
#             dataset = datasets[0]
#             aip.VideoDataset.delete(dataset.resource_name)
#             print("Deleted dataset:", dataset)
#         except Exception as e:
#             print(e)

# try:
#     if delete_pipeline and "DISPLAY_NAME" in globals():
#         pipelines = aip.PipelineJob.list(
#             filter=f"display_name={DISPLAY_NAME}", order_by="create_time"
#         )
#         pipeline = pipelines[0]
#         aip.PipelineJob.delete(pipeline.resource_name)
#         print("Deleted pipeline:", pipeline)
# except Exception as e:
#     print(e)

# if delete_bucket and "BUCKET_NAME" in globals():
#     ! gsutil rm -r $BUCKET_NAME